In [13]:
import numpy as np 
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

In [14]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Project-copy/

Mounted at /content/drive
/content/drive/MyDrive/Project-copy


In [15]:
x_train = pd.read_csv('stat3612-project-task1/X_train.csv', header=[0,1,2])
x_train.drop(columns=x_train.columns[0], inplace=True)
x_valid = pd.read_csv('stat3612-project-task1/X_valid.csv', header=[0,1,2])
x_valid.drop(columns=x_valid.columns[0], inplace=True)
x_test = pd.read_csv('stat3612-project-task1/X_test.csv', header=[0,1,2])
x_test.drop(columns=x_test.columns[0], inplace=True)
x_test
x_trainc = x_train.copy()
x_validc = x_valid.copy()
x_testc = x_test.copy()

In [16]:
y_train = pd.read_csv('stat3612-project-task2/Y_train.csv')
y_train.drop(columns=y_train.columns[0], inplace=True)
y_valid = pd.read_csv('stat3612-project-task2/Y_valid.csv')
y_valid.drop(columns=y_valid.columns[0], inplace=True)

# Random Forest

In [17]:
def refea(data, f):
    data = pd.DataFrame(data, columns = f)
    return data

def findf(data):
    ls = []
    for i in data.columns:
        ls.append(i)
    return ls
feature = findf(x_valid)

In [18]:
x_trainc = x_train.copy()
x_validc = x_valid.copy()
x_testc = x_test.copy()

y_train = pd.read_csv('stat3612-project-task2/Y_train.csv')
y_train.drop(columns=y_train.columns[0], inplace=True)
y_valid = pd.read_csv('stat3612-project-task2/Y_valid.csv')
y_valid.drop(columns=y_valid.columns[0], inplace=True)

In [19]:
from sklearn.ensemble import RandomForestRegressor
for d in range(7,8):
    crf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = d, random_state = 18) # different random states may have different model outcome
    crf.fit(x_train, y_train.values.ravel())
    fi = crf.feature_importances_
    fn = []
    for bal in [0.00019]:
        for i in range(len(feature)):
            if fi[i] >= bal:
                fn.append(feature[i])
        x_trainc = refea(x_trainc, fn)
        x_validc = refea(x_validc, fn)
        x_testc = refea(x_testc, fn)
        for depth in [17]:
            crf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = depth, random_state = 18)
            crf.fit(x_trainc, y_train.values.ravel())
            print('depth1:',d,'bal:',bal,'depth:', depth, crf.score(x_validc, y_valid))

depth1: 7 bal: 0.00019 depth: 17 0.2344504855589652


In [20]:
y_valid_df = pd.DataFrame()
y_valid_df['rf'] = crf.predict(x_validc)
y_valid_df

,rf
0,2.844764
1,4.867074
2,3.386828
3,2.639988
4,2.386071
...,...
2389,2.764175
2390,2.618433
2391,5.069033
2392,3.113983


In [21]:
yrf = crf.predict(x_testc)
ytest_df = pd.DataFrame()
ytest_df['rf'] = yrf
ytest_df

,rf
0,3.796696
1,2.841419
2,2.615795
3,2.727100
4,2.698900
...,...
4785,3.551689
4786,2.863886
4787,2.581065
4788,2.759429


# Boosting

In [22]:
x_trainc = x_train.copy()
x_validc = x_valid.copy()
x_testc = x_test.copy()

In [23]:
from sklearn.ensemble import RandomForestRegressor
for d in [8]:
    crf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = d, random_state = 18)
    crf.fit(x_train, y_train.values.ravel())
    fi = crf.feature_importances_
    fn = []
    for bal in [0.0002]:
        for i in range(len(feature)):
            if fi[i] >= bal:
                fn.append(feature[i])
        x_trainc = refea(x_trainc, fn)
        x_validc = refea(x_validc, fn)
        x_testc = refea(x_testc, fn)
        for depth in [5]:
            for cs in [0.9]:
                bst = XGBRegressor(n_estimators = 150, max_depth =depth, eval_metric = ["auc"], random_state = 18, colsample_bytree = cs)
                bst.fit(x_trainc, y_train.values.ravel())
                print('depth1:',d,'bal:',bal,'depth:', depth,'colsample:', cs, bst.score(x_validc, y_valid))

[09:03:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
depth1: 8 bal: 0.0002 depth: 5 colsample: 0.9 0.24227105143871563


In [24]:
y_valid_df['boost'] = bst.predict(x_validc)
y_valid_df

,rf,boost
0,2.844764,2.972758
1,4.867074,5.385190
2,3.386828,3.369930
3,2.639988,2.521559
4,2.386071,1.845801
...,...,...
2389,2.764175,2.848988
2390,2.618433,1.848299
2391,5.069033,5.003479
2392,3.113983,3.115679


In [25]:
ybst = bst.predict(x_testc)

ytest_df['boost'] = ybst
ytest_df

,rf,boost
0,3.796696,4.004812
1,2.841419,2.708376
2,2.615795,2.853604
3,2.727100,2.275831
4,2.698900,2.575777
...,...,...
4785,3.551689,3.705172
4786,2.863886,3.439012
4787,2.581065,2.050490
4788,2.759429,2.633265


# Linear Regression

In [26]:
x_trainc = x_train.copy()
x_validc = x_valid.copy()
x_testc = x_test.copy()
x_validc

alanine aminotransferase                                               \
                         mask                                                
                            0    1    2    3    4    5    6    7    8    9   
0                         0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
1                         0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
2                         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3                         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4                         1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2389                      1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
2390                      1.0  0.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0   
2391                      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2392                      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2393                      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

      ... white blood cell count urine                                     \
      ...          time_since_measured                                      
      ...                           14     15     16     17     18     19   
0     ...                        100.0  100.0  100.0  100.0  100.0  100.0   
1     ...                        100.0  100.0  100.0  100.0  100.0  100.0   
2     ...                        100.0  100.0  100.0  100.0  100.0  100.0   
3     ...                        100.0    0.0    1.0    2.0    3.0    4.0   
4     ...                         23.0   24.0   25.0   26.0   27.0   28.0   
...   ...                          ...    ...    ...    ...    ...    ...   
2389  ...                        152.0  153.0  154.0  155.0  156.0  157.0   
2390  ...                        176.0  177.0  178.0  179.0  180.0  181.0   
2391  ...                         14.0   15.0   16.0   17.0   18.0   19.0   
2392  ...                         38.0   39.0   40.0   41.0   42.0   43.0   
2393  ...                         62.0   63.0   64.0   65.0   66.0   67.0   

                                  
                                  
         20     21     22     23  
0     100.0  100.0  100.0  100.0  
1     100.0  100.0  100.0  100.0  
2     100.0  100.0  100.0  100.0  
3       5.0    6.0    7.0    8.0  
4      29.0   30.0   31.0   32.0  
...     ...    ...    ...    ...  
2389  158.0  159.0  160.0  161.0  
2390  182.0  183.0  184.0  185.0  
2391   20.0   21.0   22.0   23.0  
2392   44.0   45.0   46.0   47.0  
2393   68.0   69.0   70.0   71.0  

[2394 rows x 7488 columns]

In [27]:
features = ['glucose', 'creatinine', 'blood urea nitrogen', 'oxygen saturation', 'platelets', 'bicarbonate', 'diastolic blood pressure', 'heart rate', 'systolic blood pressure', 'mean blood pressure', 'respiratory rate', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'anion gap', 'prothrombin time pt', 'partial thromboplastin time', 'phosphate', 'phosphorous', 'co2 (etco2, pco2, etc.)', 'partial pressure of carbon dioxide', 'lactate', 'glascow coma scale total', 'co2', 'neutrophils', 'lymphocytes', 'monocytes', 'positive end-expiratory pressure set', 'alanine aminotransferase', 'asparate aminotransferase', 'lactic acid', 'tidal volume set', 'basophils', 'albumin', 'tidal volume spontaneous', 'central venous pressure', 'fraction inspired oxygen set', 'lactate dehydrogenase', 'fibrinogen', 'fraction inspired oxygen', 'cardiac index', 'systemic vascular resistance', 'white blood cell count urine']
fe = []
for i in features:
    for j in ['mask','mean']:
        for k in range(24):
            fe.append((i,j,str(k)))
fnew = []
for i in features:
    fnew.append(i+'_mean')
    fnew.append(i+'_mask')
    fnew.append(i+'_range')

In [28]:
def refea(data, f):
    data = pd.DataFrame(data, columns = f)
    return data

def findf(data):
    ls = []
    for i in data.columns:
        ls.append(i)
    return ls

def rerefea(data, f2, f):
    df = pd.DataFrame()
    fea = []
    for i in range(len(f2)):
        if i%3 == 0:
            df[f2[i]] = data[(f[int(i/3)],'mean',str(0))]/24
            for j in range(1,24):
                df[f2[i]] += data[(f[int(i/3)],'mean',str(j))]/24
        if i%3 == 1:
            df[f2[i]] = data[(f[int(i/3)],'mask',str(0))]
            for j in range(1,24):
                df[f2[i]] += data[(f[int(i/3)],'mask',str(j))]
        if i%3 ==2:
            for j in range(24):
                fea.append((f[int(i/3)],'mean',str(j)))
            df[f2[i]] = data[fea].max(axis=1)-data[fea].min(axis=1)
            fea = []
    return df       

def feature_eng(data, f1, f2, f):
    data = refea(data,f1)
    data = rerefea(data,f2,f)
    return data

In [29]:
x_trainc = feature_eng(x_trainc,fe,fnew,features)
x_validc = feature_eng(x_validc,fe,fnew,features)
x_testc = feature_eng(x_testc,fe,fnew,features)
x_traind = x_trainc.copy()
x_validd = x_validc.copy()
x_testd = x_testc.copy()
x_trainc

,glucose_mean,glucose_mask,glucose_range,creatinine_mean,creatinine_mask,creatinine_range,blood urea nitrogen_mean,blood urea nitrogen_mask,blood urea nitrogen_range,oxygen saturation_mean,...,fraction inspired oxygen_range,cardiac index_mean,cardiac index_mask,cardiac index_range,systemic vascular resistance_mean,systemic vascular resistance_mask,systemic vascular resistance_range,white blood cell count urine_mean,white blood cell count urine_mask,white blood cell count urine_range
0,1.418719,9.0,2.688241,0.758056,2.0,0.067749,0.771572,2.0,0.101930,-1.648680,...,0.0,-1.024557,6.0,1.516091,0.830109,6.0,2.475359,0.160067,1.0,0.0
1,0.121306,4.0,1.771077,6.011407,2.0,1.151729,1.784309,2.0,0.138996,0.708781,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,0.120647,5.0,0.774846,-0.080335,2.0,0.135498,-0.441551,2.0,0.046332,0.162653,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,-0.181780,5.0,0.458582,-0.481182,2.0,0.000000,-0.516840,2.0,0.231659,-0.127311,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,-0.224608,17.0,1.771077,-0.006940,2.0,0.270995,0.220609,2.0,0.509651,-0.181217,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16755,-0.921706,3.0,0.221385,-0.021055,3.0,0.067749,-0.194448,3.0,0.046332,-0.065223,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
16756,-0.880855,2.0,0.094879,-0.005058,2.0,0.022583,-0.363688,2.0,0.077220,0.325411,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
16757,-0.694309,8.0,0.679967,-0.009763,2.0,0.135498,0.115397,2.0,0.046332,-0.565486,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
16758,-0.056593,7.0,1.059483,-0.173489,4.0,0.541990,-0.240780,4.0,0.370655,0.052558,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,-0.303768,1.0,0.0


In [30]:
from sklearn.ensemble import RandomForestRegressor
for e in [250]:
    for d in [7]:
        crf = RandomForestRegressor(n_estimators = e, max_features = 'sqrt', max_depth = d, random_state = 18)
        crf.fit(x_trainc, y_train.values.ravel())
        print('estimators:',e,'depth:',d,crf.score(x_validc, y_valid))

estimators: 250 depth: 7 0.1702071290719841


In [31]:
def refea(data, f):
    data = pd.DataFrame(data, columns = f)
    return data

def findf(data):
    ls = []
    for i in data.columns:
        ls.append(i)
    return ls
feature = findf(x_validc)

In [32]:
fj = crf.feature_importances_
fm = []
for i in range(len(feature)):
    if fj[i] >= 0.015:
        fm.append(feature[i])
x_trainn = x_trainc.copy()
x_validn = x_validc.copy()
x_testn = x_testc.copy()
x_trainn = refea(x_trainn, fm)
x_validn = refea(x_validn, fm)
x_testn = refea(x_testn,fm)
x_trainn

,blood urea nitrogen_mean,respiratory rate_mean,"co2 (etco2, pco2, etc.)_mask","co2 (etco2, pco2, etc.)_range",partial pressure of carbon dioxide_mask,partial pressure of carbon dioxide_range,glascow coma scale total_mean,positive end-expiratory pressure set_mean,positive end-expiratory pressure set_mask,tidal volume set_mean,tidal volume set_mask,fraction inspired oxygen set_mask
0,0.771572,-1.043440,8.0,1.265078,8.0,0.633450,-2.125838,0.886201,7.0,0.440628,7.0,8.0
1,1.784309,-1.018978,0.0,0.000000,0.0,0.000000,0.793194,0.000000,0.0,0.000000,0.0,0.0
2,-0.441551,-0.855653,3.0,0.798997,3.0,0.950175,-1.567788,0.682097,7.0,1.139394,7.0,7.0
3,-0.516840,-0.058747,0.0,0.000000,0.0,0.000000,0.728804,0.000000,0.0,0.000000,0.0,0.0
4,0.220609,-0.726826,6.0,2.796489,6.0,2.217075,-1.015104,-0.338421,7.0,0.530469,7.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16755,-0.194448,-0.630107,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
16756,-0.363688,-0.004140,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
16757,0.115397,-0.290387,5.0,0.599248,5.0,1.055750,0.000000,0.000000,0.0,0.000000,0.0,0.0
16758,-0.240780,1.028054,1.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [33]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree=2)
x_trainn = poly_reg.fit_transform(x_trainn)
x_validn = poly_reg.fit_transform(x_validn)
x_testn = poly_reg.fit_transform(x_testn)


In [34]:
x_t = x_traind.values
x_v = x_validd.values
x_te = x_testd.values
x_trainn = pd.DataFrame(x_trainn)
x_validn = pd.DataFrame(x_validn)
x_testn = pd.DataFrame(x_testn)

x_t = pd.DataFrame(x_t)
x_v = pd.DataFrame(x_v)
x_te = pd.DataFrame(x_te)

In [35]:
x1 = pd.concat([x_t,x_trainn],axis=1)
x2 = pd.concat([x_v,x_validn],axis=1)
x3 = pd.concat([x_te,x_testn],axis=1)

In [36]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
pol_reg = LinearRegression()
pol_reg.fit(x1, y_train)
pol_reg.score(x2, y_valid)

0.18738693399756368

In [37]:
y_valid_df['reg'] = pol_reg.predict(x2)

y_valid_df

,rf,boost,reg
0,2.844764,2.972758,3.236945
1,4.867074,5.385190,5.214999
2,3.386828,3.369930,3.636313
3,2.639988,2.521559,2.792403
4,2.386071,1.845801,1.166057
...,...,...,...
2389,2.764175,2.848988,2.745557
2390,2.618433,1.848299,1.748091
2391,5.069033,5.003479,4.690912
2392,3.113983,3.115679,3.142132


In [38]:
ytest_df['reg'] = pol_reg.predict(x3)
ytest_df

,rf,boost,reg
0,3.796696,4.004812,4.019255
1,2.841419,2.708376,3.035204
2,2.615795,2.853604,2.521316
3,2.727100,2.275831,3.053396
4,2.698900,2.575777,2.431617
...,...,...,...
4785,3.551689,3.705172,4.422061
4786,2.863886,3.439012,2.589066
4787,2.581065,2.050490,2.650108
4788,2.759429,2.633265,3.068608


## linear regression of 3 models

In [39]:
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_valid, y_valid_df['rf'])))
print(np.sqrt(mean_squared_error(y_valid, y_valid_df['boost'])))
print(np.sqrt(mean_squared_error(y_valid, y_valid_df['reg'])))

1.7449237873362924
1.7359881654856613
1.797760032221125


In [40]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(y_valid_df, y_valid)

In [41]:
print(reg.coef_)

[[0.40743842 0.57090124 0.1609359 ]]


In [42]:
# y_rf_depth17 = reg.predict(ytest_df)

In [43]:
lrtuning = reg.predict(ytest_df)

In [44]:
lrtuning

array([[3.97477459],
       [2.68705185],
       [2.59533167],
       ...,
       [2.14340988],
       [2.61614051],
       [3.44916643]])

In [ ]:
df = pd.DataFrame(lrtuning)
df.to_excel('lrtuning.xlsx')